# TODO: implement the do_pre_compute() function that do pre-compute according to the metadata

In [1]:
import pandas as pd

df = pd.read_csv('pre_compute_metadata.csv')

In [2]:
def do_pre_compute(record):
    pass
    

for record in df.to_records():
    do_pre_compute(record)

# Note: temporal and spatial resampling and file encoding

In [3]:
import xarray

ds = xarray.open_dataset('/data/era5/raw/2m_temperature/2m_temperature-2014.nc')
ds

<xarray.Dataset> Size: 73GB
Dimensions:    (longitude: 1440, latitude: 721, time: 8760)
Coordinates:
  * longitude  (longitude) float32 6kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 70kB 2014-01-01 ... 2014-12-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float64 73GB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-03 18:48:50 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [4]:
# Use resample to get different temporal resolution
ds_temporal = ds.resample(time='D').min()
ds_temporal

<xarray.Dataset> Size: 3GB
Dimensions:    (time: 365, latitude: 721, longitude: 1440)
Coordinates:
  * longitude  (longitude) float32 6kB -180.0 -179.8 -179.5 ... 179.5 179.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 3kB 2014-01-01 2014-01-02 ... 2014-12-31
Data variables:
    t2m        (time, latitude, longitude) float64 3GB 250.5 250.5 ... 244.6
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-03 18:48:50 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [5]:
# Use sel to get different spatial resolution
step = 1
ds_spatial = ds.sel(latitude=range(90, -90 - step, -step), longitude=range(-180, 180, step))
ds_spatial

<xarray.Dataset> Size: 5GB
Dimensions:    (longitude: 360, latitude: 181, time: 8760)
Coordinates:
  * longitude  (longitude) float32 1kB -180.0 -179.0 -178.0 ... 178.0 179.0
  * latitude   (latitude) float32 724B 90.0 89.0 88.0 87.0 ... -88.0 -89.0 -90.0
  * time       (time) datetime64[ns] 70kB 2014-01-01 ... 2014-12-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float64 5GB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-03 18:48:50 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [9]:
# use encoding to convert float values to int values, to save storage
encoding = ds['t2m'].encoding
ds_temporal['t2m'].encoding = encoding
ds_spatial['t2m'].encoding = encoding
encoding

{'dtype': dtype('int16'),
 'source': '/data/era5/raw/2m_temperature/2m_temperature-2014.nc',
 'original_shape': (8760, 721, 1440),
 'missing_value': np.int16(-32767),
 '_FillValue': np.int16(-32767),
 'scale_factor': np.float64(0.002007091327153996),
 'add_offset': np.float64(259.20215960130906)}